ALS를 이용한 추천

In [ ]:
import numpy as np
import pandas as pd
from scipy import sparse
from data_manager import merge_data
from metric_calculator import precision_recall_at_k
from pathlib import Path
import pandas as pd
from scipy import sparse
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import train_test_split
from tqdm import tqdm
import pickle

%load_ext autoreload
%autoreload 2

In [ ]:
# # 데이터 전처리 및 저장
# DATA_DIR = "2023q4_form13f/"
# merge_data(DATA_DIR)

In [ ]:
DATA_DIR = 'data/'

df = pd.read_parquet(f"{DATA_DIR}holdings_data.parquet")

# 인덱스 매핑
cik2id  = {cik: i for i, cik in enumerate(df["CIK"].unique())}
cusip2id = {cusip: j for j, cusip in enumerate(df["CUSIP"].unique())}

# 기관을 사용자로 상품은 주식으로 취급
df["user_id"] = df["CIK"].map(cik2id) 
df["item_id"] = df["CUSIP"].map(cusip2id) 

n_users = len(cik2id)
n_items = len(cusip2id)

rows = df["item_id"].values
cols = df["user_id"].values
data = df["hold"].values

item_user_mat = sparse.coo_matrix((data, (rows, cols)), 
                                  shape=(n_items, n_users)).tocsr()

train_mat, test_mat = train_test_split(item_user_mat, train_percentage=0.8, random_state=42)

als = AlternatingLeastSquares(
    factors = 64,
    regularization = 0.1,
    iterations = 100,
    random_state = 42,
    use_gpu = False
)

als.fit(train_mat)

with open(f"{DATA_DIR}als_model.pkl", "wb") as f:
    pickle.dump({"model": als, "cik2id": cik2id, "cusip2id": cusip2id}, f, protocol=pickle.HIGHEST_PROTOCOL)